In [1]:
# Meshing
from meshing import process
            
process(data_path="data/models", output_path="data/meshes", sort=False, rescale=False, 
            filter_level=0, precision=[0.95, 0.04], subfolders=False)

ModuleNotFoundError: No module named 'OCC'

In [19]:
# Create charuco board
import cv2
from cv2 import aruco
import numpy as np
from PIL import Image
import os
os.chdir("/home/sebastian/Projects/SRB/Package/scanner-sim")

# dic = aruco.getPredefinedDictionary(aruco.DICT_5X5_250)
# board = aruco.CharucoBoard_create(25, 18, 0.015, 0.01167, dic)
# d = board.draw((1800, 2500), 10, 1)
# im = Image.fromarray(d)
# im.save("data/objects/charuco.png")


def create_charuco(n=25, m=18, size=15, draw=True, draw_scale=1):
    aruco_dict = aruco.Dictionary_get(aruco.DICT_5X5_250)
    board = aruco.CharucoBoard_create(n, m, size, size * 7 / 9, aruco_dict)
    
    img = board.draw((3750, 2700), 10, 10)

    m_pos, m_ids, _ = cv2.aruco.detectMarkers(img, aruco_dict)

    if len(m_pos) > 0:
        count, c_pos, c_ids = cv2.aruco.interpolateCornersCharuco(m_pos, m_ids, img, board)
    else:
        count, c_pos, c_ids = 0, None, None

    if count:
        img_points, obj_points = np.array(c_pos).reshape((-1, 2)), board.chessboardCorners[c_ids].reshape((-1, 3))
        ids = c_ids.ravel()
    else:
        img_points, obj_points, ids = None, None, None

    if draw:
        img = aruco.drawDetectedMarkers(img, np.array(m_pos), m_ids)
        img = aruco.drawDetectedCornersCharuco(img, np.array(c_pos), c_ids, cornerColor=(0, 255, 0))


    return (img_points, obj_points, ids), img

ids, img = create_charuco(draw=False)

im = Image.fromarray(img)
im.save("data/objects/charuco.png")

In [20]:
def create_checkerboard_ply(fname="cb", shape=[25, 18], size=20, color1=[0, 0, 0], color2=[255, 255, 255]):
    header = "ply\nformat ascii 1.0\nelement vertex %i\nproperty float x\nproperty float y\nproperty float z\nproperty float nx\nproperty float ny\nproperty float nz\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nelement face %i\nproperty list uchar uint vertex_indices\nend_header\n"
    faces = []
    vertices = []
    x2 = shape[0] / 2 * size
    z2 = shape[1] / 2 * size
    vc = 0
    for r in range(shape[1]):
        for c in range(shape[0]):
            if r%2 == 0 and c%2 == 0 or (r+1)%2 == 0 and (c+1)%2 == 0:
                col = color1
            else:
                col = color2
            v0 = ((c * size) - x2, 0.0, (r * size) - z2)
            v1 = (((c+1) * size) - x2, 0.0, (r * size) - z2)
            v2 = ((c * size) - x2, 0.0, ((r+1) * size) - z2)
            v3 = (((c+1) * size) - x2, 0.0, ((r+1) * size) - z2)
            vertices.append("%0.6f %0.6f %0.6f 0.0 -1.0 0.0 %i %i %i"%(v0[0], v0[1], v0[2], col[0], col[1], col[2]))
            vertices.append("%0.6f %0.6f %0.6f 0.0 -1.0 0.0 %i %i %i"%(v1[0], v1[1], v1[2], col[0], col[1], col[2]))
            vertices.append("%0.6f %0.6f %0.6f 0.0 -1.0 0.0 %i %i %i"%(v2[0], v2[1], v2[2], col[0], col[1], col[2]))
            vertices.append("%0.6f %0.6f %0.6f 0.0 -1.0 0.0 %i %i %i"%(v3[0], v3[1], v3[2], col[0], col[1], col[2]))

            faces.append("4 %i %i %i %i"%(vc, vc+1, vc+3, vc+2))
            vc += 4

    with open(fname+"%i_%i.ply"%(shape[0], shape[1]), "w") as fi:
        fi.write(header%(len(vertices), len(faces)))
        for v in vertices:
            fi.write(v+"\n")
        for f in faces:
            fi.write(f+"\n")
    print("Created checkerboard")
    
create_checkerboard_ply()

Created checkerboard


In [33]:
# Preprocessing/Create gray patterns

import patterns as pa
import matplotlib


# Initialize textures
def init_textures(tex_size=[1920, 1080]):
    gray = pa.genboard_graycode(proj_resolution=tex_size)
    textures_h = np.swapaxes(gray[0][:, :], 0, 2)
    textures_v = np.swapaxes(gray[1][:, :], 0, 2)
    white = np.ones(tex_size[::-1], dtype=np.uint8)*255
    black = np.ones(tex_size[::-1], dtype=np.uint8)

    patterns = [white, black]
    for t in textures_v:
        patterns.append(t)
        patterns.append(255-t)
    for t in textures_h:
        patterns.append(t)
        patterns.append(255-t)
        
    for p_i, p in enumerate(patterns):
        pat = Image.fromarray(p)
        pat.save("rendering/patterns/gray/default/img_%02i.png"%p_i)

textures = init_textures()



def load_images(prefix, pars, mm=[1, 39], typ="256"):
    images = []
    for i in range(mm[0], mm[1]):
        filename_r = "%s/%s/%s/%s_%02i.png"%(pars["root_folder"], pars["img_folder"], typ, prefix, i)
        im = matplotlib.image.imread(filename_r)
        img = rgb2gray(im)*255

        #print(img.shape, img.dtype, np.min(img), np.max(img))
        images.append(img)
    return images